<a href="https://colab.research.google.com/github/ikanx101/AppSheet-Sales/blob/main/AppSheet_MDS_2025.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# _AppSheet Converter_ 2025 untuk ___MDS___

_Converter_ ini dibuat untuk mengubah data hasil _input_ __AppSheet: MDS__ ke bentuk tabular yang lebih _excel-user-friendly_.

Cara pakai:

1. Klik tombol `Connect` pada pojok kanan atas untuk menyalakan _server cloud Google Colab_.
1. _Upload_ _file_ yang dibutuhkan, yakni:
    - Data __AppSheet: MDS__.
1. Isi _form_ sesuai dengan nama _files_ tersebut dan __nama__ ___sheet___ yang hendak diproses.
1. Klik `Runtime` >> `Run all`.
1. Tunggu hingga proses selesai (sampai muncul pesan `PROSES SELESAI`). Kira-kira akan memakan waktu 5-10 menit.
1. Akan muncul _file_ baru bernama `MDS Converted.xlsx`. Silakan di-_download_.

---

Catatan: Karena yang kita gunakan saat ini adalah _server_ gratisnya Google, jadi kita harus selalu _install_ beberapa _packages_ setiap kali _run_ skrip ini. Konsekuensinya _runtime_-nya lebih lama.

_Last Update_: 7 Maret 2025

# New Section

In [ ]:
                #@title Masukkan Nama Files

rm(list=ls())
gc()

nama_file_utama <- "MDS MODERN.xlsx" #@param {type:"string"}
nama_sheet <- "Rekap" #@param {type:"string"}

install.packages("reshape2")
install.packages("janitor")
install.packages("openxlsx")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,658499,35.2,1454452,77.7,1454452,77.7
Vcells,1225814,9.4,8388608,64.0,1975093,15.1


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘plyr’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependency ‘snakecase’


Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)



In [ ]:
# @title
# libraries yang dibutuhkan
library(dplyr)
library(tidyr)
library(openxlsx)
library(janitor)
library(readxl)

# ambil data mds
df = read_excel(nama_file_utama,sheet = nama_sheet)

nama_kol    = colnames(df)
batas_akhir = which(nama_kol == "Klasifikasi Customer")

id_penting  = 1:batas_akhir
kol_penting = nama_kol[id_penting]

df_1 = df |> select(all_of(kol_penting))

kol_penting = nama_kol[2:batas_akhir]
df_2 =
  df |>
  select(-all_of(kol_penting)) |>
  reshape2::melt(id.vars = "ID") |>
  mutate(variable = as.character(variable)) |>
  filter(value != "N/A") |>
  filter(value != "Tidak Jual") |>
  rename("Products"      = variable,
         "Status Produk" = value)

df_final =
  df_1 |>
  merge(df_2) |>
  janitor::clean_names() |>
  mutate(amount = 0) |>
  mutate(brand = case_when(
    grepl("hi lo",products,ignore.case = T) ~ "Hi Lo",
    grepl("l-men",products,ignore.case = T) ~ "L-Men",
    grepl("ns",products,ignore.case = T) ~ "NutriSari",
    grepl("ts",products,ignore.case = T) ~ "Tropicana Slim",
    grepl("hi lo",products,ignore.case = T) ~ "Hi Lo"
  )) |>
  select(tanggal,nama_mds_spg,region,klasifikasi_customer,customer_name,customer_code,
         products,amount,status_produk,brand,divisi)

colnames(df_final) = c("Submission Date","Pic","Area","Klasifikasi Customer","Customer Name",
                       "Customer Code","Products","Amount","Status Produk","Brand","DIV")

openxlsx::write.xlsx(df_final,file = "MDS Converted.xlsx",overwrite = T)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘janitor’


The following objects are masked from ‘package:stats’:

    chisq.test, fisher.test




In [ ]:
#@title Status
print("PROSES SELESAI")
print("file hasil konversi tersimpan dalam nama: MDS Converted.xlsx")

[1] "PROSES SELESAI"
[1] "file hasil konversi tersimpan dalam nama: MDS Converted.xlsx"
